# Object localization.

#### imports and GPU check

In [2]:
import torch, mmdetection.mmdet as mmdet
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

print(mmdet.__version__)

gpu_available = torch.cuda.is_available()
print(f"GPU available: {gpu_available}")

if gpu_available:
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_19:00:59_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  2.0.1
3.2.0
GPU available: True
GPU name: NVIDIA RTX A4000


#### enviroment inforamtion

In [3]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdetection.mmdet as mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

sys.platform: win32
Python: 3.8.17 (default, Jul  5 2023, 20:35:33) [MSC v.1916 64 bit (AMD64)]
CUDA available: True
numpy_random_seed: 2147483648
GPU 0: NVIDIA RTX A4000
CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.7
NVCC: Cuda compilation tools, release 11.7, V11.7.64
MSVC: n/a, reason: fileno
PyTorch: 2.0.1
PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=comput

#### download checkpoints (for finetuning, if needed)

In [ ]:
# !mkdir ./checkpoints
# !mim download mmdet --config rtmdet-ins_l_8xb32-300e_coco --dest ./checkpoints
# !mim download mmdet --config mask-rcnn_r50-caffe_fpn_ms-poly-3x_coco --dest ./checkpoints

#### convert dataset to coco (bbox detection only)

In [ ]:
import os.path as osp
import mmcv
import mmengine
import cv2
import os
import json
import numpy as np
from tqdm import tqdm
import pycocotools.mask as maskUtils
import random



def is_clockwise(contour):
    value = 0
    num = len(contour)
    for i, point in enumerate(contour):
        p1 = contour[i]
        if i < num - 1:
            p2 = contour[i + 1]
        else:
            p2 = contour[0]
        value += (p2[0][0] - p1[0][0]) * (p2[0][1] + p1[0][1]);
    return value < 0

def get_merge_point_idx(contour1, contour2):
    idx1 = 0
    idx2 = 0
    distance_min = -1
    for i, p1 in enumerate(contour1):
        for j, p2 in enumerate(contour2):
            distance = pow(p2[0][0] - p1[0][0], 2) + pow(p2[0][1] - p1[0][1], 2);
            if distance_min < 0:
                distance_min = distance
                idx1 = i
                idx2 = j
            elif distance < distance_min:
                distance_min = distance
                idx1 = i
                idx2 = j
    return idx1, idx2

def merge_contours(contour1, contour2, idx1, idx2):
    contour = []
    for i in list(range(0, idx1 + 1)):
        contour.append(contour1[i])
    for i in list(range(idx2, len(contour2))):
        contour.append(contour2[i])
    for i in list(range(0, idx2 + 1)):
        contour.append(contour2[i])
    for i in list(range(idx1, len(contour1))):
        contour.append(contour1[i])
    contour = np.array(contour)
    return contour

def merge_with_parent(contour_parent, contour):
    if not is_clockwise(contour_parent):
        contour_parent = contour_parent[::-1]
    if is_clockwise(contour):
        contour = contour[::-1]
    idx1, idx2 = get_merge_point_idx(contour_parent, contour)
    return merge_contours(contour_parent, contour, idx1, idx2)

def mask2polygon(image):
    contours, hierarchies = cv2.findContours(image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    contours_approx = []
    polygons = []
    for contour in contours:
        epsilon = 0.001 * cv2.arcLength(contour, True)
        contour_approx = cv2.approxPolyDP(contour, epsilon, True)
        contours_approx.append(contour_approx)

    contours_parent = []
    for i, contour in enumerate(contours_approx):
        parent_idx = hierarchies[0][i][3]
        if parent_idx < 0 and len(contour) >= 3:
            contours_parent.append(contour)
        else:
            contours_parent.append([])

    for i, contour in enumerate(contours_approx):
        parent_idx = hierarchies[0][i][3]
        if parent_idx >= 0 and len(contour) >= 3:
            contour_parent = contours_parent[parent_idx]
            if len(contour_parent) == 0:
                continue
            contours_parent[parent_idx] = merge_with_parent(contour_parent, contour)

    contours_parent_tmp = []
    for contour in contours_parent:
        if len(contour) == 0:
            continue
        contours_parent_tmp.append(contour)

    polygons = []
    for contour in contours_parent_tmp:
        polygon = contour.flatten().tolist()
        polygons.append(polygon)
    return polygons, contours

def rle2polygon(segmentation, mask):
    if isinstance(segmentation["counts"], list):
        segmentation = mask.frPyObjects(segmentation, *segmentation["size"])
    m = mask.decode(segmentation)
    m[m > 0] = 255
    polygons = mask2polygon(m)
    return polygons

def get_files_ending_with_ext(dir, ext='.jpg'):
    collection = {}
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(ext):
                collection[os.path.basename(file)[:-4]] = os.path.join(root, file)
    return collection

def vis_side_by_side(img, mask, title):
    fig, ax = plt.subplots(1,2)
    fig.suptitle(title)
    ax[0].imshow(img)
    ax[1].imshow(mask*50)
    plt.show()
    return

def hierarchical_cnt_to_yolo_fmt(contours, h, w):
    contours = [np.array(cnt) for cnt in contours]
    mask = np.zeros((h, w)).astype(np.uint8)
    cv2.drawContours(mask, contours, -1, 1, thickness=cv2.FILLED)
    ## convert mask to poly and preserve hierarchy in YOLO format
    modified_polys, original_polys = mask2polygon(mask)
    return modified_polys, original_polys



def create_annotation_dict(ann_idx,
                           ann_obj_count,
                           ann_current_category_id,
                           ann_abs_bbox, ann_abs_w,
                           ann_abs_h):
    """
    Create a dictionary structure for annotation data.

    Parameters:
    - ann_idx (int): The index of the annotation.
    - ann_obj_count (int): A count of the annotation objects.
    - ann_current_category_id (int): The ID of the current category.
    - ann_abs_bbox (list): The bounding box.
    - ann_abs_w (int): The width.
    - ann_abs_h (int): The height.

    Returns:
    dict: A dictionary containing the formatted annotation data.
    """

    data_anno = dict(
        image_id=ann_idx,
        id=ann_obj_count,
        category_id=ann_current_category_id,
        bbox=ann_abs_bbox,
        area=ann_abs_w * ann_abs_h,
        iscrowd=0
    )

    return data_anno



def seg_to_rle(contours_list, height, width):

    # create an empty mask of the same size as the original image
    mask = np.zeros((height, width), dtype=np.uint8)

    # fill the contours in the mask
    cv2.drawContours(mask, contours_list, -1, 1, thickness=cv2.FILLED)

    ## convert the binary mask to uncompressed RLE format
    rle_uncompressed = maskUtils.encode(np.asfortranarray(mask))
    rle_size = rle_uncompressed['size']

    ## convert the RLE format to a compressed string
    rle = maskUtils.encode(np.asfortranarray(mask))['counts'].decode('utf-8')

    return rle, rle_size


def create_mock_container_mask(bbox):
    x, y, w, h = bbox

    # calculate reduction % on each side
    reduction = int(w * 0.15)

    # create a smaller contour with the lower horizontal line shorter by reduction % from each side and centered
    mock_contour = [
        [x + 1, y + 1],  # Top-left
        [x + w - 2, y + 1],  # Top-right
        [x + w - reduction - 2, y + h - 2], # bottom-right (reduction % from the right)
        [x + reduction + 1, y + h - 2] # bottom-left (reduction % from the left)
    ]
    
    flat_mock_contour = [[item for sublist in mock_contour for item in sublist]]
    
    return flat_mock_contour


def convert_dataset_to_coco(jsons_dir, out_file, subset):

    """
    Convert dataset annotations in JSON format to COCO format.

    Parameters:
    - jsons_dir (str): The directory path containing JSON files of dataset annotations.
    - out_file (str): The file path where the converted annotations in COCO format will be saved.
    - subset (str): train/val subset identifier 

    Side effects:
    - Writes a file: Outputs a file containing the annotations in COCO format at `out_file`.
    """

    category_ids_dict = {
        'cars': 0,
        'dogs': 1,
        'cats': 2,
    }

    # set the seed
    random.seed(42)

    # get the json files
    json_files = []
    for root, dirs, files in os.walk(jsons_dir):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    json_files = list(set(json_files))

    # shuffle the list of JSON files
    random.shuffle(json_files)

    print('found {} json files'.format(len(json_files)))

    annotations = []
    images = []
    obj_count = 0

    for idx, json_file_path in tqdm(enumerate(json_files), desc="Processing JSON files", ncols=100, total=len(json_files)):

        with open(json_file_path) as f:
            data = json.load(f)

        filename = os.path.join(data["image"]["file_path"], data["image"]["file_name"])

        height, width = data['image']['height'], data['image']['width']

        images.append(dict(
            id=idx,
            file_name=filename,
            height=height,
            width=width))

        assert isinstance(data["annotations"]["bbox"], list) or isinstance(data["annotations"]["container_bbox"], list), 'Images must have at least one bbox.'

        ###########################################################
        ## get annotations for object
        ###########################################################
        if (subset not in ['val', 'train']) or not isinstance(data["annotations"]["bbox"], list):            
            pass
        else:
            assert len(data["annotations"]["bbox"]) == 4, 'Invalid bbox format.'
            
            # get category ID (species class)
            species_name = str(data["annotations"]["species"]).lower()
            if species_name in category_ids_dict:
                current_category_id = category_ids_dict[species_name]
            else:
                raise Exception(f'Unknown species {data["annotations"]["species"]} encountered.')

            # get bbox
            abs_x, abs_y, abs_w, abs_h = data['annotations']["bbox"]
            abs_bbox = [abs_x, abs_y, abs_w, abs_h]


            # create the annotation dictionary
            data_anno = create_annotation_dict(ann_idx=idx,
                                               ann_obj_count=obj_count,
                                               ann_current_category_id=current_category_id,
                                               ann_abs_bbox=abs_bbox,
                                               ann_abs_w=abs_w,
                                               ann_abs_h=abs_h,
                                              )
            annotations.append(data_anno)
            obj_count += 1


        ###########################################################
        ## get annotations for object 2
        ###########################################################
        # make sure there is bbox
        if (subset not in ['val', 'train']) or (not isinstance(data["annotations"]["container_bbox"], list)):
            pass
        else:
            assert len(data["annotations"]["container_bbox"]) == 4, 'Invalid bbox format.'

            # get category ID
            current_category_id = category_ids_dict['container']

            # get bbox
            abs_x, abs_y, abs_w, abs_h = data['annotations']["container_bbox"]
            abs_bbox = [abs_x, abs_y, abs_w, abs_h]


            # create the annotation dictionary
            data_anno = create_annotation_dict(ann_idx=idx,
                                               ann_obj_count=obj_count,
                                               ann_current_category_id=current_category_id,
                                               ann_abs_bbox=abs_bbox,
                                               ann_abs_w=abs_w,
                                               ann_abs_h=abs_h,
                                              )
            annotations.append(data_anno)
            obj_count += 1

            
    # get the categories
    categories = [{'id': id, 'name': name} for name, id in category_ids_dict.items()]

    # create the coco format json
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=categories
    )

    # save the coco format json
    mmengine.dump(coco_format_json, out_file)
    
    
    
    print('WARNING - this loader is edited to not check segmentation json validation =accepted, since this exp is for bbox detection')


#### set up date dirs

In [ ]:
import os

image_data_root = 'G:/Datasets/images'
json_data_root = 'G:/Datasets/jsons'

train_dataset_path = os.path.join(json_data_root, 'train')
val_dataset_path = os.path.join(json_data_root, 'val')

train_export_path = os.path.join(json_data_root, 'coco_train.json')
val_export_path = os.path.join(json_data_root, 'coco_val.json')

#### convert data to coco format

In [ ]:
convert_dataset_to_coco(jsons_dir = train_dataset_path,
                        out_file = train_export_path,
                        subset = 'train')

convert_dataset_to_coco(jsons_dir = val_dataset_path,
                        out_file = val_export_path,
                        subset = 'val')


#### Checking the label corresponding to the instance split ID after the data format conversion is complete

In [ ]:
from pycocotools.coco import COCO

# path to load the COCO annotation file
annotation_file = val_export_path

# initialise the COCO object
coco = COCO(annotation_file)

# get all category tags and corresponding category IDs
categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'] for cat in categories}

# print all category IDs and corresponding category names
for category_id, category_name in category_id_to_name.items():
    print(f"Category ID: {category_id}, Category Name: {category_name}")

#### visualize the dataset to make sure things are as expected

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import mmcv
import os

# initialize the COCO object
coco = COCO(annotation_file)

# load an image and its annotations
for img_id in range(10):
    img_info = coco.loadImgs(img_id)[0]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    # load the image from a local file
    img_path = os.path.join(image_data_root, img_info['file_name'])
    img = mmcv.imread(img_path)

    # create a matplotlib figure and axis
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # draw bounding boxes
    for ann in anns:
        bbox = ann['bbox']
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    
    print('img_path', img_path)
    plt.axis('off')
    plt.title(img_path)
    plt.show()


#### initiate config

In [ ]:
from mmengine import Config
from mmengine.runner import set_random_seed
from pprint import pprint

# config file
cfg = Config.fromfile('./mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_amp-1x_coco.py')

# pre-training weight paths.
cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_fp16_1x_coco_20200204-d4dc1471.pth'

# working folder
cfg.work_dir = './work_dir'

# max epochs
cfg.max_epochs = 100

# batch size
cfg.train_dataloader.batch_size = 4

# dataloader num workers
cfg.train_dataloader.num_workers = 8

# The original learning rate (LR) is set for 8-GPU training. We divide it by 8 since we only use one GPU.
change_in_batch_size = cfg.train_dataloader.batch_size / 2 # added this to account for our batch size, default in mmdet is 2 per gpu, when using batch of 4, means we have size bigger by a factor of 2 times.
cfg.optim_wrapper.optimizer.lr = (0.02 / 8) * change_in_batch_size

# metainfo 
cfg.metainfo = {
    'classes': ('cars', 'dogs', 'cats'),
    'palette': [
        (220, 20, 60),    # RGB color for class 'cars' (red-ish)
        (65, 105, 225),   # RGB color for class 'dogs' (blue-ish)
        (34, 139, 34),    # RGB color for class 'cats' (green-ish)
    ]
}

# data folder
cfg.data_root = image_data_root

# train json file path
cfg.train_dataloader.dataset.ann_file = './dataset/train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root

# train image file path
cfg.train_dataloader.dataset.data_prefix.img = '' # we dont need prefix

# update metainfo
cfg.train_dataloader.dataset.metainfo = cfg.metainfo

# valid json file path
cfg.val_dataloader.dataset.ann_file = './dataset/val.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root

# valid image file path
cfg.val_dataloader.dataset.data_prefix.img = ''
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader = cfg.val_dataloader

# valid evaluator json file path
cfg.val_evaluator.ann_file = cfg.data_root + './dataset/val.json'

cfg.val_evaluator.metric = ['bbox']

cfg.test_evaluator = cfg.val_evaluator

cfg.test_evaluator.classwise = True

# model weights are saved every interval, up to two weights are saved at the same time, and the saving strategy is auto.
cfg.default_hooks.checkpoint = dict(type='CheckpointHook', interval=3, max_keep_ckpts=2, save_best='auto')

# interval of reporting indicators
cfg.default_hooks.logger.interval = 10

cfg.train_cfg.max_epochs = cfg.max_epochs

# evaluation of the model begins at epoch n
cfg.train_cfg.val_begin = 10

# evaluate the model every n epochs 
cfg.train_cfg.val_interval = 10 

# dataset loader :
cfg.train_dataloader.dataset = cfg.train_dataloader.dataset

# fixed random number seed
set_random_seed(0, deterministic=False)

# we can also use tensorboard to log the training process (optional)
cfg.visualizer.vis_backends.append({"type":'TensorboardVisBackend'})

# Modify num classes of the model in box head and mask head
cfg.model.roi_head.bbox_head.num_classes = 9

config=f'./mmdetection/configs/custom_detector_config.py'
with open(config, 'w') as f:
    f.write(cfg.pretty_text)

pprint(vars(cfg))


#### train func

In [ ]:
# Copyright (c) OpenMMLab. All rights reserved.
import logging
import os
import os.path as osp
from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
from mmdetection.mmdet.utils import setup_cache_size_limit_of_dynamo

# Instead of parsing args, we will use this configuration class
class ConfigArgs:
    def __init__(self, config_path):
        self.config = config_path
        self.work_dir = None
        self.amp = False
        self.auto_scale_lr = False
        self.resume = None
        self.cfg_options = None
        self.launcher = 'none'
        self.local_rank = 0


def trigger_training(config_path):
    args = ConfigArgs(config_path)
    
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    # Reduce the number of repeated compilations and improve training speed.
    setup_cache_size_limit_of_dynamo()

    # load config
    cfg = Config.fromfile(args.config)
    cfg.launcher = args.launcher
    if args.cfg_options is not None:
        cfg.merge_from_dict(args.cfg_options)

    # determine work_dir
    if args.work_dir is not None:
        cfg.work_dir = args.work_dir
    elif cfg.get('work_dir', None) is None:
        ## if not in cfg
        cfg.work_dir = osp.join('./work_dir',
                                osp.splitext(osp.basename(args.config))[0])

    # enable automatic-mixed-precision training
    if args.amp is True:
        optim_wrapper = cfg.optim_wrapper.type
        if optim_wrapper == 'AmpOptimWrapper':
            print_log(
                'AMP training is already enabled in your config.',
                logger='current',
                level=logging.WARNING)
        else:
            assert optim_wrapper == 'OptimWrapper', (
                '`--amp` is only supported when the optimizer wrapper type is '
                f'`OptimWrapper` but got {optim_wrapper}.')
            cfg.optim_wrapper.type = 'AmpOptimWrapper'
            cfg.optim_wrapper.loss_scale = 'dynamic'

    # enable automatically scaling LR
    if args.auto_scale_lr:
        if 'auto_scale_lr' in cfg and \
                'enable' in cfg.auto_scale_lr and \
                'base_batch_size' in cfg.auto_scale_lr:
            cfg.auto_scale_lr.enable = True
        else:
            raise RuntimeError('Can not find "auto_scale_lr" or '
                               '"auto_scale_lr.enable" or '
                               '"auto_scale_lr.base_batch_size" in your'
                               ' configuration file.')

    # resume is determined in this priority: resume from > auto_resume
    if args.resume == 'auto':
        cfg.resume = True
        cfg.load_from = None
    elif args.resume is not None:
        cfg.resume = True
        cfg.load_from = args.resume

    
    # build the runner from config
    if 'runner_type' not in cfg:
        runner = Runner.from_cfg(cfg)
    else:
        runner = RUNNERS.build(cfg)

    # start training
    runner.train()


#### run training

In [ ]:
trigger_training(config)

#### test func

In [ ]:
# Copyright (c) OpenMMLab. All rights reserved.
import os
import os.path as osp
import warnings
from copy import deepcopy
from mmengine import ConfigDict
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.engine.hooks.utils import trigger_visualization_hook
from mmdet.evaluation import DumpDetResults
from mmdet.registry import RUNNERS
from mmdet.utils import setup_cache_size_limit_of_dynamo

# Configuration class
class ConfigArgs:
    def __init__(self, config_path, checkpoint_path, out):
        self.config = config_path
        self.checkpoint = checkpoint_path
        self.work_dir = None
        self.out = out #None
        self.show = False
        self.show_dir = None
        self.wait_time = 2
        self.cfg_options = None
        self.launcher = 'none'
        self.tta = False
        self.local_rank = 0

def trigger_testing(config_path, checkpoint_path, out):
    args = ConfigArgs(config_path, checkpoint_path, out)
    
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    # reduce the number of repeated compilations and improve testing speed.
    setup_cache_size_limit_of_dynamo()

    # load config
    cfg = Config.fromfile(args.config)
    cfg.launcher = args.launcher
    if args.cfg_options is not None:
        cfg.merge_from_dict(args.cfg_options)

    # determine work_dir
    if args.work_dir:
        cfg.work_dir = args.work_dir
    elif not cfg.get('work_dir'):
        cfg.work_dir = osp.join('./work_dir', osp.splitext(osp.basename(args.config))[0])

    cfg.load_from = args.checkpoint

    if args.show or args.show_dir:
        cfg = trigger_visualization_hook(cfg, args)

    if args.tta:
        if 'tta_model' not in cfg:
            warnings.warn('Cannot find ``tta_model`` in config, we will set it as default.')
            cfg.tta_model = dict(type='DetTTAModel', tta_cfg=dict(nms=dict(type='nms', iou_threshold=0.5), max_per_img=100))
        if 'tta_pipeline' not in cfg:
            warnings.warn('Cannot find ``tta_pipeline`` in config, we will set it as default.')
            test_data_cfg = cfg.test_dataloader.dataset
            while 'dataset' in test_data_cfg:
                test_data_cfg = test_data_cfg['dataset']
            cfg.tta_pipeline = deepcopy(test_data_cfg.pipeline)
            flip_tta = dict(
                type='TestTimeAug',
                transforms=[
                    [dict(type='RandomFlip', prob=1.), dict(type='RandomFlip', prob=0.)],
                    [dict(type='PackDetInputs', meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor', 'flip', 'flip_direction'))],
                ])
            cfg.tta_pipeline[-1] = flip_tta
        cfg.model = ConfigDict(**cfg.tta_model, module=cfg.model)
        cfg.test_dataloader.dataset.pipeline = cfg.tta_pipeline

    # build the runner from config
    if 'runner_type' not in cfg:
        runner = Runner.from_cfg(cfg)
    else:
        runner = RUNNERS.build(cfg)

    if args.out:
        assert args.out.endswith(('.pkl', '.pickle')), 'The dump file must be a pkl file.'
        runner.test_evaluator.metrics.append(DumpDetResults(out_file_path=args.out))

    # start testing
    runner.test()


#### trigger test

In [ ]:
trigger_testing(config, "./work_dir/best_coco_bbox_mAP_epoch_90.pth", out = './results_file.pkl')